In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

txt_file =  input("txt dosyası: ")
with open(txt_file, "r", encoding="utf-8") as file:
    for line in file:
        list_for_excel = list()
        line = line.strip()
        line = line.split(",")
        excel_file = line[0]        # kaydedilecek excel dosyasının ismi
        starting_page = line[1]     #başlangıç sayfası
        ending_page = line[2]       # bitiş sayfası
        f_website_url = line[3]     # site linki
        
        starting_page = int(starting_page)
        ending_page = int(ending_page)

        try:
            # ALINAN LİNKTEKİ SAYFALARA GİDEREK İLAN LİNKLERİNİ ÇEKEN VE LİSTEYE ATAN KISIM
            # bota karşı önlem alınmış olabilir o sebeple headers ekledik.
            headers = {
                "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36", 
                "X-Amzn-Trace-Id": "Root=1-6209086d-6b68c17b4f73e9d6174b5736"
            }

            urls = list()   # toplanan ilan linklerini atacağımız liste
            url_counter = 1     #toplanan ilan linklerini sayan sayaç
            while(starting_page <= ending_page):
                print("\n------------------------\n")
                print("SAYFA: " + str(starting_page))
                website_url = f"{f_website_url}?page={str(starting_page)}"  #url sonuna sayfa parametresi ekledik
                print(website_url)
                print("\n------------------------\n")

                r = requests.get(website_url, headers=headers)  #websiteye request attık
                soup = BeautifulSoup(r.content, "lxml")     # dönen response'u parçaladık

                adv_urls = soup.find_all("a", attrs={"class":"card-link"})  # ilan linklerini tutan class
                for i in adv_urls:
                    adv_url = "https://www.hepsiemlak.com" + i.get("href")  # url sonuna ilan pathini ekledik
                    urls.append(adv_url)
                    print(str(url_counter) + ") " + adv_url)
                    url_counter += 1
                    
                starting_page += 1      # sayfa numarasını bir artırdık
                website_url = f_website_url     #linki ilk haline eşitledik ki sonuna yeni sayfa parametesini ekleyebilelim
                time.sleep(1.2)     # sayfadan sayfaya geöerken her request arası 1.2 sn bekletiyoruz

            print("\nİLAN LİNKLERİ ALINDI, VERİ ÇEKMEYE BAŞLIYORUZ...\n")
            time.sleep(3)


            # ÇEKİLEN İLAN LİNKLERİNE TEKER TEKER GİDİP VERİSİNİ ÇEKECECEK KISIM
            counter = 1
            for url in urls:
                headers = {
                "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36", 
                "X-Amzn-Trace-Id": "Root=1-6209086d-6b68c17b4f73e9d6174b5736"
                }

                r = requests.get(url, headers=headers)
                soup = BeautifulSoup(r.content, "lxml")
                ilan_url = url

                print("\n-----------------------------------\n" + str(counter))
                print("Link: " + ilan_url)
                try:
                    name = soup.find("div", attrs={"class":"firm-link fontRB"}).getText().strip().replace("\n", " ").replace("               ", " ").split("Mesleki")
                    name = name[0]
                except:
                    name = soup.find("div", attrs={"class":"owner-firm-name fontRB mb10"}).getText().strip()
                print("Satıcı: " + name)

                title = soup.find("h1", attrs={"class":"fontRB"}).getText().strip()
                print("İlan Başlığı: " + title)

                try:
                    seller = soup.find("div", attrs={"class":"firm-link"}).getText().strip().replace("\n", " ").replace("               ", " ").split("Mesleki")
                    seller = seller[0]
                except:
                    seller = soup.find("span", attrs={"class":"owner-member-type mr10"}).getText().strip()
                print("Emlak Şirketi: " + seller)

                price = soup.find("p", attrs={"class":"fontRB fz24 price"}).getText().strip()
                print("Fiyat: " + price)

                try:
                    state1 = soup.find("ul", attrs={"class":"short-info-list"})
                    state2 = state1.find_all("li")
                    short_info_list = list()
                    for info in state2:
                        info = info.getText().strip()
                        short_info_list.append(info)

                    sehir = short_info_list[0]
                    ilce = short_info_list[1]
                    mahalle = short_info_list[2]
                except:
                    sehir = "-"
                    ilce = "-"
                    mahalle = "-"

                ilan_no = "-"
                ilan_durumu = "-"
                konut_sekli = "-"
                oda_salon_sayisi = "-"
                brut_net_m2 = "-"
                bulundugu_kat = "-"
                bina_yasi = "-"
                isinma_tipi = "-"
                kat_sayisi = "-"
                metrekare = "-"
                kredi_durumu = "-"
                esya_durumu  = "-"
                yapi_tipi = "-"
                yapi_durumu = "-"
                aidat  = "-"
                cephe = "-"
                tapu_durumu = "-"
                kira_getirisi = "-"
                depozito = "-"
                yakit_tipi = "-"

                st1 = soup.find_all("ul", attrs={"class":"adv-info-list"})
                for i in st1:
                    st2 = i.find_all("li", attrs={"class":"spec-item"})
                    for j in st2:
                        detail = j.getText().strip()

                        if("İlan no" in detail):
                            ilan_no = detail.lstrip("İlan no ")
                            
                        if("İlan Durumu" in detail):
                            ilan_durumu = detail.lstrip("İlan Durumu ")
                            
                        if("Konut Şekli" in detail):
                            konut_sekli = detail.lstrip("Konut Şekli ")
                            
                        if("Bulunduğu Kat" in detail):
                            bulundugu_kat = detail.lstrip("Bulunduğu Kat ")
                            
                        if("Bina Yaşı" in detail):
                            bina_yasi = detail.lstrip("Bina Yaşı ")
                            
                        if("Isınma Tipi" in detail):
                            isinma_tipi = detail.lstrip("Isınma Tipi ")
                            
                        if("Kat Sayısı" in detail):
                            kat_sayisi = detail.lstrip("Kat Sayısı ")
                            
                        if("Oda + Salon Sayısı" in detail):
                            oda_salon_sayisi = detail.lstrip("Oda + Salon Sayısı ")

                        if("Brüt / Net M2" in detail):
                            brut_net_m2 = detail.lstrip("Brüt / Net M2 ").strip()

                        if("Metrekare" in detail and "m2" in detail):
                            metrekare = detail.lstrip("Metrekare ")
                        
                        if("Krediye Uygunluk" in detail):
                            kredi_durumu = detail.split(" ")
                            kredi_durumu = kredi_durumu[2]
                            
                        if("Eşya Durumu" in detail):
                            esya_durumu = detail.split("Durumu ")
                            esya_durumu = esya_durumu[1]
                            
                        if("Yapı Tipi" in detail):
                            yapi_tipi = detail.lstrip("Yapı Tipi ")
                            
                        if("Yapının Durumu" in detail):
                            yapi_durumu = detail.lstrip("Yapının Durumu ")
                            
                        if("Aidat" in detail):
                            aidat = detail.strip("Aidat ")
                            
                        if("Cephe" in detail):
                            cephe = detail.lstrip("Cephe ")
                        
                        if("Tapu Durumu" in detail):
                            tapu_durumu = detail.lstrip("Tapu Durumu ")
                        
                        if("Kira Getirisi" in detail):
                            kira_getirisi = detail.lstrip("Kira Getirisi ")
                            
                        if("Depozito" in detail):
                            depozito = detail.lstrip("Depozito ")
                            
                        if("Yakıt Tipi" in detail):
                            yakit_tipi = detail.lstrip("Yakıt Tipi ")

                # İLAN VERİLERİ ÇEKİLİRKEN CMD ÜZERİNDE AKACAK OLAN VERİLER
                print("İlan no: " + ilan_no)
                print("Şehir: " + sehir)
                print("İlçe: " + ilce)
                print("Mahalle: " + mahalle)
                print("İlan Durumu: " + ilan_durumu)
                print("Konut Şekli: " + konut_sekli)
                print("Bulunduğu kat: " + bulundugu_kat)
                print("Bina Yaşı: " + bina_yasi)
                print("Isınma Tipi: " + isinma_tipi)
                print("Kat Sayısı: " + kat_sayisi)
                print("Oda + Salon Sayısı: " + oda_salon_sayisi)
                print("Brüt / Net M2: " + brut_net_m2)
                print("Metrekare: " + metrekare)
                print("Krediye Uygunluk: " + kredi_durumu)
                print("Eşya Durumu: " + esya_durumu)
                print("Yapı Tipi: " + yapi_tipi)
                print("Yapının Durumu: " + yapi_durumu)
                print("Aidat: " + aidat)
                print("Cephe: " + cephe)
                print("Tapu Durumu: " + tapu_durumu)
                print("Kira Getirisi: " + kira_getirisi)
                print("Depozito: " + depozito)
                print("Yakıt Tipi: " + yakit_tipi)

                # İLAN VERİLERİNİ DICTIONARY TİPİNDE TUTYORUZ
                advert_infos = {
                    "İlan no":ilan_no,
                    "Konut Şekli":konut_sekli,
                    "Satıcı":name,
                    "İlan Başlığı":title,
                    "Fiyat":price,
                    "Link":ilan_url,
                    "İlan Durumu":ilan_durumu,
                    "Şehir":sehir,
                    "İlçe":ilce,
                    "Mahalle":mahalle,
                    "Emlakçı":seller,         
                    "Bulunduğu kat":bulundugu_kat,
                    "Bina Yaşı":bina_yasi,
                    "Isınma Tipi":isinma_tipi,
                    "Kat Sayısı":kat_sayisi,
                    "Oda + Salon Sayısı":oda_salon_sayisi,
                    "Brüt / Net M2":brut_net_m2,
                    "Metrekare":metrekare,
                    "Krediye Uygunluk":kredi_durumu,
                    "Eşya Durumu":esya_durumu,
                    "Yapı Tipi":yapi_tipi,
                    "Yapının Durumu":yapi_durumu,
                    "Aidat":aidat,
                    "Cephe":cephe,
                    "Tapu Durumu":tapu_durumu,
                    "Kira Getirisi":kira_getirisi,
                    "Depozito":depozito,
                    "Yakıt Tipi":yakit_tipi,
                }
                list_for_excel.append(advert_infos)
                print("\n-------------------------------------\n")

                counter += 1
                time.sleep(1.2)
        except:
            print("HATA ALINDI PROGRAMDAN ÇIKILIYOR....")
        finally:
            # ÇEKİLEN VERİLERİ EXCEL DOSYASINA ÇEVİREN KISIM
            print("\nVeri çekme işlemi tamamlandı...\nÇekilen veriler Excel dosyasına dönüştürülüyor...")
            file_name = excel_file + ".xlsx"
            df_data = pd.DataFrame(list_for_excel)
            df_data.to_excel(file_name, index = False)
            print("\n******************************************\nVeriler Excel dosyasına dönüştürüldü...\n******************************************\n")
            print("PROGRAM SONLANDI")

txt dosyası:  linkler.txt



------------------------

SAYFA: 1
https://www.hepsiemlak.com/istanbul-satilik?page=1

------------------------


------------------------

SAYFA: 2
https://www.hepsiemlak.com/istanbul-satilik?page=2

------------------------


İLAN LİNKLERİ ALINDI, VERİ ÇEKMEYE BAŞLIYORUZ...


Veri çekme işlemi tamamlandı...
Çekilen veriler Excel dosyasına dönüştürülüyor...

******************************************
Veriler Excel dosyasına dönüştürüldü...
******************************************

PROGRAM SONLANDI


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Test sayfası
test_url = "https://www.hepsiemlak.com/istanbul-satilik?page=1"

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-6209086d-6b68c17b4f73e9d6174b5736"
}

try:
    # 1. Sayfaya gidiyoruz
    print("Sayfa isteniyor:", test_url)
    response = requests.get(test_url, headers=headers)
    soup = BeautifulSoup(response.content, "lxml")

    # 2. İlan linklerini alıyoruz
    ilan_linkleri = soup.find_all("a", attrs={"class":"card-link"})

    if not ilan_linkleri:
        print("‼️ Hiç ilan linki bulunamadı. Class adı değişmiş olabilir.")
    else:
        print(f"✅ {len(ilan_linkleri)} ilan bulundu. İlk 2 tanesi işlenecek...\n")

    urls = ["https://www.hepsiemlak.com" + i.get("href") for i in ilan_linkleri[:2]]
    
    ilan_listesi = []

    # 3. Her ilan linkine gidiyoruz
    for index, ilan_url in enumerate(urls, 1):
        print(f"\n--- İlan {index} ---\n{ilan_url}")
        ilan_resp = requests.get(ilan_url, headers=headers)
        ilan_soup = BeautifulSoup(ilan_resp.content, "lxml")

        try:
            title = ilan_soup.find("h1", class_="fontRB").get_text(strip=True)
        except:
            title = "Bulunamadı"

        try:
            price = ilan_soup.find("p", class_="fontRB fz24 price").get_text(strip=True)
        except:
            price = "Bulunamadı"

        try:
            location_info = ilan_soup.find("ul", class_="short-info-list").find_all("li")
            sehir = location_info[0].get_text(strip=True)
            ilce = location_info[1].get_text(strip=True)
            mahalle = location_info[2].get_text(strip=True)
        except:
            sehir, ilce, mahalle = "-", "-", "-"

        print("Başlık:", title)
        print("Fiyat:", price)
        print("Konum:", sehir, "/", ilce, "/", mahalle)

        ilan_listesi.append({
            "Link": ilan_url,
            "Başlık": title,
            "Fiyat": price,
            "Şehir": sehir,
            "İlçe": ilce,
            "Mahalle": mahalle
        })

        time.sleep(1.5)  # güvenlik için

    # 4. Excel'e kaydet
    df = pd.DataFrame(ilan_listesi)
    df.to_excel("test_sonuclari.xlsx", index=False)
    print("\n✅ test_sonuclari.xlsx dosyası başarıyla kaydedildi!")

except Exception as e:
    print("🚨 Hata oluştu:", e)


Sayfa isteniyor: https://www.hepsiemlak.com/istanbul-satilik?page=1
‼️ Hiç ilan linki bulunamadı. Class adı değişmiş olabilir.

✅ test_sonuclari.xlsx dosyası başarıyla kaydedildi!


In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time


chrome_options = Options()
# chrome_options.add_argument("--headless")  # bu satırı kapat!
driver = webdriver.Chrome(options=chrome_options)

driver.get("https://www.hepsiemlak.com/istanbul-satilik?page=1")
time.sleep(3)

ilanlar = driver.find_elements(By.CSS_SELECTOR, "a.card-link")
print(f"{len(ilanlar)} ilan bulundu")

for i, ilan in enumerate(ilanlar[:2], 1):
    print(f"{i}) {ilan.get_attribute('href')}")

driver.quit()


19 ilan bulundu
1) https://www.hepsiemlak.com/istanbul-beylikduzu-cumhuriyet-satilik/daire/132292-884
2) https://www.hepsiemlak.com/istanbul-basaksehir-bahcesehir-2-kisim-satilik/daire/132292-863


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

# ====== AYARLAR ======
BASLANGIC_SAYFA = 1
BITIS_SAYFA = 3
BASE_URL = "https://www.hepsiemlak.com/istanbul-satilik"
EXCEL_DOSYASI = "istanbul_satilik_selenium.xlsx"
# ======================

# Selenium ayarları (GÖRÜNÜR tarayıcı için headless kapalı!)
options = Options()
# options.add_argument("--headless")  # Gerekirse açarsın ama şu an kapalı kalmalı
driver = webdriver.Chrome(options=options)

ilan_listesi = []

for sayfa in range(BASLANGIC_SAYFA, BITIS_SAYFA + 1):
    print(f"\n📄 Sayfa {sayfa} işleniyor...")
    url = f"{BASE_URL}?page={sayfa}"
    driver.get(url)
    time.sleep(4)  # Sayfanın yüklenmesini bekle

    ilanlar = driver.find_elements(By.CSS_SELECTOR, "a.card-link")
    print(f"{len(ilanlar)} ilan bulundu.")

    for i, ilan in enumerate(ilanlar[:20], 1):  # İstersen tüm ilanlar için: remove [:20]
        ilan_url = ilan.get_attribute("href")
        print(f"\n🔗 {i}. ilan: {ilan_url}")

        # Detay sayfasına git
        driver.get(ilan_url)
        time.sleep(3)

        try:
            baslik = driver.find_element(By.TAG_NAME, "h1").text.strip()
        except:
            baslik = "Bulunamadı"

        try:
            fiyat = driver.find_element(By.CSS_SELECTOR, "p.price").text.strip()
        except:
            fiyat = "Bulunamadı"

        try:
            konum_elemanlari = driver.find_elements(By.CSS_SELECTOR, "ul.short-info-list li")
            sehir = konum_elemanlari[0].text.strip() if len(konum_elemanlari) > 0 else "-"
            ilce = konum_elemanlari[1].text.strip() if len(konum_elemanlari) > 1 else "-"
            mahalle = konum_elemanlari[2].text.strip() if len(konum_elemanlari) > 2 else "-"
        except:
            sehir, ilce, mahalle = "-", "-", "-"

        print("🏷️ Başlık:", baslik)
        print("💰 Fiyat:", fiyat)
        print("📍 Konum:", sehir, "/", ilce, "/", mahalle)

        ilan_listesi.append({
            "Link": ilan_url,
            "Başlık": baslik,
            "Fiyat": fiyat,
            "Şehir": sehir,
            "İlçe": ilce,
            "Mahalle": mahalle
        })

        time.sleep(2)

# Excel'e yaz
df = pd.DataFrame(ilan_listesi)
df.to_excel(EXCEL_DOSYASI, index=False)
print(f"\n✅ Excel dosyası kaydedildi: {EXCEL_DOSYASI}")

driver.quit()



📄 Sayfa 1 işleniyor...
19 ilan bulundu.

🔗 1. ilan: https://www.hepsiemlak.com/istanbul-beylikduzu-cumhuriyet-satilik/daire/132292-884
🏷️ Başlık: ÖNCÜDEN METROBÜSE 10DK MESAFEDE FIRSAT 2+1 LÜX SATILIK DAİRE !
💰 Fiyat: 2.860.000 TL
📍 Konum: - / - / -


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=134.0.6998.166); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6D5404C25+3179557]
	(No symbol) [0x00007FF6D50688A0]
	(No symbol) [0x00007FF6D4EF91CA]
	(No symbol) [0x00007FF6D4F0F5C4]
	(No symbol) [0x00007FF6D4F0E093]
	(No symbol) [0x00007FF6D4F018C1]
	(No symbol) [0x00007FF6D4F01A21]
	(No symbol) [0x00007FF6D4EFF632]
	(No symbol) [0x00007FF6D4F03F11]
	(No symbol) [0x00007FF6D4FA1652]
	(No symbol) [0x00007FF6D4F77C2A]
	(No symbol) [0x00007FF6D4FA02F3]
	(No symbol) [0x00007FF6D4F77A03]
	(No symbol) [0x00007FF6D4F406D0]
	(No symbol) [0x00007FF6D4F41983]
	GetHandleVerifier [0x00007FF6D54667CD+3579853]
	GetHandleVerifier [0x00007FF6D547D1D2+3672530]
	GetHandleVerifier [0x00007FF6D5472153+3627347]
	GetHandleVerifier [0x00007FF6D51D09

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Kullanıcıdan txt dosyası al
txt_file = input("txt dosyası: ")

# Chrome ayarları
options = Options()
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# txt içeriğini oku
with open(txt_file, "r", encoding="utf-8") as file:
    for line in file:
        ilanlar_listesi = []
        line = line.strip().split(",")
        excel_adi = line[0]
        baslangic = int(line[1])
        bitis = int(line[2])
        base_url = line[3]

        print(f"\n⏳ {excel_adi} için sayfalar geziliyor...")

        # SAYFALARDA GEZ, İLAN LİNKLERİNİ TOPLA
        ilan_linkleri = []
        for sayfa in range(baslangic, bitis + 1):
            sayfa_url = f"{base_url}?page={sayfa}"
            print(f"🔎 Sayfa {sayfa}: {sayfa_url}")
            driver.get(sayfa_url)

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "a.card-link"))
                )
                kartlar = driver.find_elements(By.CSS_SELECTOR, "a.card-link")
                linkler = [k.get_attribute("href") for k in kartlar]
                ilan_linkleri.extend(linkler)
                print(f"✅ {len(linkler)} ilan bulundu.")
            except:
                print("❌ Sayfa yüklenemedi veya ilan bulunamadı.")

        print(f"\n📦 Toplam {len(ilan_linkleri)} ilan linki bulundu. Veriler çekiliyor...")

        # HER İLAN SAYFASINA GİT, VERİLERİ AL
        for i, ilan_url in enumerate(ilan_linkleri, 1):
            print(f"\n📄 {i}. ilan işleniyor: {ilan_url}")
            driver.get(ilan_url)

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "h1"))
                )
            except:
                print("❌ İlan sayfası yüklenmedi.")
                continue

            try:
                baslik = driver.find_element(By.TAG_NAME, "h1").text.strip()
            except:
                baslik = "-"

            try:
                fiyat = driver.find_element(By.CSS_SELECTOR, "p.price").text.strip()
            except:
                fiyat = "-"

            try:
                konum = driver.find_elements(By.CSS_SELECTOR, "ul.detail-info-location")
                sehir = konum[0].text.strip() if len(konum) > 0 else "-"
                ilce = konum[1].text.strip() if len(konum) > 1 else "-"
                mahalle = konum[2].text.strip() if len(konum) > 2 else "-"
            except:
                sehir, ilce, mahalle = "-", "-", "-"

            ilanlar_listesi.append({
                "Başlık": baslik,
                "Fiyat": fiyat,
                "Şehir": sehir,
                "İlçe": ilce,
                "Mahalle": mahalle,
                "Link": ilan_url
            })

        # EXCEL'E KAYDET
        df = pd.DataFrame(ilanlar_listesi)
        dosya_adi = excel_adi + ".xlsx"
        df.to_excel(dosya_adi, index=False)
        print(f"\n✅ Veriler Excel dosyasına kaydedildi: {dosya_adi}")

driver.quit()
print("\n🎉 Tüm işlemler tamamlandı.")


txt dosyası:  linkler.txt



⏳ istanbul_satilik için sayfalar geziliyor...
🔎 Sayfa 1: https://www.hepsiemlak.com/istanbul-satilik?page=1
❌ Sayfa yüklenemedi veya ilan bulunamadı.
🔎 Sayfa 2: https://www.hepsiemlak.com/istanbul-satilik?page=2


InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF6D5404C25+3179557]
	(No symbol) [0x00007FF6D50688A0]
	(No symbol) [0x00007FF6D4EF8FFC]
	(No symbol) [0x00007FF6D4F3F8DF]
	(No symbol) [0x00007FF6D4F77AF2]
	(No symbol) [0x00007FF6D4F7244E]
	(No symbol) [0x00007FF6D4F714F9]
	(No symbol) [0x00007FF6D4EC55E5]
	GetHandleVerifier [0x00007FF6D54667CD+3579853]
	GetHandleVerifier [0x00007FF6D547D1D2+3672530]
	GetHandleVerifier [0x00007FF6D5472153+3627347]
	GetHandleVerifier [0x00007FF6D51D092A+868650]
	(No symbol) [0x00007FF6D5072FFF]
	(No symbol) [0x00007FF6D4EC41FF]
	GetHandleVerifier [0x00007FF6D54EFA28+4141608]
	BaseThreadInitThunk [0x00007FFBFEC9E8D7+23]
	RtlUserThreadStart [0x00007FFBFF7FBF6C+44]


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Kullanıcıdan txt dosyası al
txt_file = input("txt dosyası: ")

# Chrome ayarları ve kamuflaj
options = Options()
# options.add_argument("--headless")  # Gerekirse aç
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
        Object.defineProperty(navigator, 'webdriver', {
          get: () => undefined
        })
    """
})

# txt içeriğini oku
with open(txt_file, "r", encoding="utf-8") as file:
    for line in file:
        ilanlar_listesi = []
        line = line.strip().split(",")
        excel_adi = line[0]
        baslangic = int(line[1])
        bitis = int(line[2])
        base_url = line[3]

        print(f"\n⏳ {excel_adi} için sayfalar geziliyor...")

        ilan_linkleri = []
        for sayfa in range(baslangic, bitis + 1):
            sayfa_url = f"{base_url}?page={sayfa}"
            print(f"🔎 Sayfa {sayfa}: {sayfa_url}")
            driver.get(sayfa_url)

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "a.card-link"))
                )
                kartlar = driver.find_elements(By.CSS_SELECTOR, "a.card-link")
                linkler = [k.get_attribute("href") for k in kartlar]
                ilan_linkleri.extend(linkler)
                print(f"✅ {len(linkler)} ilan bulundu.")
            except:
                print("❌ Sayfa yüklenemedi veya ilan bulunamadı.")

        print(f"\n📦 Toplam {len(ilan_linkleri)} ilan linki bulundu. Veriler çekiliyor...")

        for i, ilan_url in enumerate(ilan_linkleri, 1):
            print(f"\n📄 {i}. ilan işleniyor: {ilan_url}")
            driver.get(ilan_url)

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "h1"))
                )
            except:
                print("❌ İlan sayfası yüklenmedi.")
                continue

            try:
                baslik = driver.find_element(By.TAG_NAME, "h1").text.strip()
            except:
                baslik = "-"

            try:
                fiyat = driver.find_element(By.CSS_SELECTOR, "p.price").text.strip()
            except:
                fiyat = "-"

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "ul.detail-info-location li"))
                )
                konum = driver.find_elements(By.CSS_SELECTOR, "ul.detail-info-location li")
                sehir = konum[0].text.strip() if len(konum) > 0 else "-"
                ilce = konum[1].text.strip() if len(konum) > 1 else "-"
                mahalle = konum[2].text.strip() if len(konum) > 2 else "-"
            except:
                sehir, ilce, mahalle = "-", "-", "-"


        

            ilanlar_listesi.append({
                "Başlık": baslik,
                "Fiyat": fiyat,
                "Şehir": sehir,
                "İlçe": ilce,
                "Mahalle": mahalle,
            })

        df = pd.DataFrame(ilanlar_listesi)
        dosya_adi = excel_adi + ".xlsx"
        df.to_excel(dosya_adi, index=False)
        print(f"\n✅ Veriler Excel dosyasına kaydedildi: {dosya_adi}")

driver.quit()
print("\n🎉 Tüm işlemler tamamlandı.")


ModuleNotFoundError: No module named 'selenium'

In [5]:
pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.4 MB 508.0 kB/s eta 0:00:18
   -- ------------------------------------- 0.5/9.4 MB 508.0 kB/s eta 0:00:18
   --- ------------------------------------ 0.8/9.4 MB 558.9 kB/s eta 0:00:16
   ---- ----------------------------------- 1.0/9.4 MB 662.0 kB/s eta 0:00:13
   ----- ---------------------------------- 1.3/9.4 MB 737.4 kB/s eta 0:00:11
   ------ --------------------------------- 1.6/9.4 MB 806.6 kB/s eta 0:00:10
   ------- -------------------------------- 1.8/9.4 MB 867.5 kB/s eta 0:00:09
   -------- ----------------------------

In [13]:
pip install undetected-chromedriver

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47130 sha256=26ebe6e320108cf799fc75aeb2d32e3c9502d13bae7d406fef6c060ea5f0b933
  Stored in directory: c:\users\bedir\appdata\local\pip\cache\wheels\c4\f1\aa\9de6cf276210554d91e9c0526864563e850a428c5e76da4914
Successfully built undetected-chromedriver
Note: you may need to restart the kernel to use updated packages.


In [19]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random

# Kullanıcıdan txt dosyası al
txt_file = input("txt dosyası: ")

# Chrome başlat (uc ile)
driver = uc.Chrome()

# txt dosyasındaki her satır için işlemleri yap
with open(txt_file, "r", encoding="utf-8") as file:
    for line in file:
        ilanlar_listesi = []
        line = line.strip().split(",")
        excel_adi = line[0]
        baslangic = int(line[1])
        bitis = int(line[2])
        base_url = line[3]

        print(f"\n⏳ {excel_adi} için sayfalar geziliyor...")

        ilan_linkleri = []
        for sayfa in range(baslangic, bitis + 1):
            sayfa_url = f"{base_url}?page={sayfa}"
            print(f"🔎 Sayfa {sayfa}: {sayfa_url}")
            driver.get(sayfa_url)

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "a.card-link"))
                )
                kartlar = driver.find_elements(By.CSS_SELECTOR, "a.card-link")
                linkler = [k.get_attribute("href") for k in kartlar]
                ilan_linkleri.extend(linkler)
                print(f"✅ {len(linkler)} ilan bulundu.")
            except:
                print("❌ Sayfa yüklenemedi veya ilan bulunamadı.")

            time.sleep(random.uniform(2.5, 5.0))  # İnsan gibi davranmak için

        print(f"\n📦 Toplam {len(ilan_linkleri)} ilan linki bulundu. Veriler çekiliyor...")

        for i, ilan_url in enumerate(ilan_linkleri, 1):
            print(f"\n📄 {i}. ilan işleniyor: {ilan_url}")
            driver.get(ilan_url)

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "h1"))
                )
            except:
                print("❌ İlan sayfası yüklenmedi.")
                continue

            try:
                baslik = driver.find_element(By.TAG_NAME, "h1").text.strip()
            except:
                baslik = "-"

            try:
                fiyat = driver.find_element(By.CSS_SELECTOR, "p.price").text.strip()
            except:
                fiyat = "-"

            # Konum bilgileri
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "ul.detail-info-location li"))
                )
                konum = driver.find_elements(By.CSS_SELECTOR, "ul.detail-info-location li")
                sehir = konum[0].text.strip() if len(konum) > 0 else "-"
                ilce = konum[1].text.strip() if len(konum) > 1 else "-"
                mahalle = konum[2].text.strip() if len(konum) > 2 else "-"
            except:
                sehir, ilce, mahalle = "-", "-", "-"

            # Teknik bilgiler
            konut_tipi = oda_sayisi = brut_net = banyo_sayisi = kat = kat_sayisi = bina_yasi = isinma = esya = yapi = yapi_durumu = kredi = aidat = tapu = kira = depozito = yakit = cephe = "-"

            try:
                teknik_liste = driver.find_elements(By.CSS_SELECTOR, "ul.adv-info-list li.spec-item")
                for item in teknik_liste:
                    text = item.text.strip()
                    if "Konut Tipi" in text:
                        konut_tipi = text.replace("Konut Tipi", "").strip()
                    elif "Oda Sayısı" in text:
                        oda_sayisi = text.replace("Oda Sayısı", "").strip()
                    elif "Brüt / Net M2" in text:
                        brut_net = text.replace("Brüt / Net M2", "").strip()
                    elif "Banyo Sayısı" in text:
                        banyo_sayisi = text.replace("Banyo Sayısı", "").strip()
                    elif "Bulunduğu Kat" in text:
                        kat = text.replace("Bulunduğu Kat", "").strip()
                    elif "Kat Sayısı" in text:
                        kat_sayisi = text.replace("Kat Sayısı", "").strip()
                    elif "Bina Yaşı" in text:
                        bina_yasi = text.replace("Bina Yaşı", "").strip()
                    elif "Isınma Tipi" in text:
                        isinma = text.replace("Isınma Tipi", "").strip()
                    elif "Eşya Durumu" in text:
                        esya = text.replace("Eşya Durumu", "").strip()
                    elif "Yapı Tipi" in text:
                        yapi = text.replace("Yapı Tipi", "").strip()
                    elif "Yapının Durumu" in text:
                        yapi_durumu = text.replace("Yapının Durumu", "").strip()
                    elif "Krediye Uygunluk" in text:
                        kredi = text.replace("Krediye Uygunluk", "").strip()
                    elif "Aidat" in text:
                        aidat = text.replace("Aidat", "").strip()
                    elif "Tapu Durumu" in text:
                        tapu = text.replace("Tapu Durumu", "").strip()
                    elif "Kira Getirisi" in text:
                        kira = text.replace("Kira Getirisi", "").strip()
                    elif "Depozito" in text:
                        depozito = text.replace("Depozito", "").strip()
                    elif "Yakıt Tipi" in text:
                        yakit = text.replace("Yakıt Tipi", "").strip()
                    elif "Cephe" in text:
                        cephe = text.replace("Cephe", "").strip()
            except:
                print("⚠️ Teknik bilgiler alınamadı.")

            ilanlar_listesi.append({
                "Başlık": baslik,
                "Fiyat": fiyat,
                "Şehir": sehir,
                "İlçe": ilce,
                "Mahalle": mahalle,
                "Konut Tipi": konut_tipi,
                "Oda Sayısı": oda_sayisi,
                "Brüt / Net M2": brut_net,
                "Banyo Sayısı": banyo_sayisi,
                "Kat": kat,
                "Kat Sayısı": kat_sayisi,
                "Bina Yaşı": bina_yasi,
                "Isınma Tipi": isinma,
                "Eşya Durumu": esya,
                "Yapı Tipi": yapi,
                "Yapının Durumu": yapi_durumu,
                "Krediye Uygunluk": kredi,
                "Aidat": aidat,
                "Tapu Durumu": tapu,
                "Kira Getirisi": kira,
                "Depozito": depozito,
                "Yakıt Tipi": yakit,
                "Cephe": cephe,
                "Link": ilan_url
            })

            time.sleep(random.uniform(2.5, 5.0))  # İlanlar arası bekleme

        df = pd.DataFrame(ilanlar_listesi)
        dosya_adi = excel_adi + ".xlsx"
        df.to_excel(dosya_adi, index=False)
        print(f"\n✅ Veriler Excel dosyasına kaydedildi: {dosya_adi}")

driver.quit()
print("\n🎉 Tüm işlemler tamamlandı.")



txt dosyası:  linkler.txt



⏳ istanbulsatilik_bolum1 için sayfalar geziliyor...
🔎 Sayfa 1: https://www.hepsiemlak.com/istanbul-satilik?page=1
✅ 19 ilan bulundu.
🔎 Sayfa 2: https://www.hepsiemlak.com/istanbul-satilik?page=2
✅ 19 ilan bulundu.
🔎 Sayfa 3: https://www.hepsiemlak.com/istanbul-satilik?page=3
✅ 19 ilan bulundu.
🔎 Sayfa 4: https://www.hepsiemlak.com/istanbul-satilik?page=4
✅ 19 ilan bulundu.
🔎 Sayfa 5: https://www.hepsiemlak.com/istanbul-satilik?page=5
✅ 19 ilan bulundu.
🔎 Sayfa 6: https://www.hepsiemlak.com/istanbul-satilik?page=6
✅ 19 ilan bulundu.
🔎 Sayfa 7: https://www.hepsiemlak.com/istanbul-satilik?page=7
✅ 19 ilan bulundu.
🔎 Sayfa 8: https://www.hepsiemlak.com/istanbul-satilik?page=8
✅ 19 ilan bulundu.
🔎 Sayfa 9: https://www.hepsiemlak.com/istanbul-satilik?page=9
✅ 19 ilan bulundu.
🔎 Sayfa 10: https://www.hepsiemlak.com/istanbul-satilik?page=10
❌ Sayfa yüklenemedi veya ilan bulunamadı.

📦 Toplam 171 ilan linki bulundu. Veriler çekiliyor...

📄 1. ilan işleniyor: https://www.hepsiemlak.com/istanbul

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=134.0.6998.166)
Stacktrace:
	GetHandleVerifier [0x0124C7F3+24435]
	(No symbol) [0x011D2074]
	(No symbol) [0x010A06E3]
	(No symbol) [0x0108FEC0]
	(No symbol) [0x010ADFDF]
	(No symbol) [0x011146AF]
	(No symbol) [0x0112EB19]
	(No symbol) [0x0110D5B6]
	(No symbol) [0x010DC54F]
	(No symbol) [0x010DD894]
	GetHandleVerifier [0x015570A3+3213347]
	GetHandleVerifier [0x0156B0C9+3295305]
	GetHandleVerifier [0x0156558C+3271948]
	GetHandleVerifier [0x012E7360+658144]
	(No symbol) [0x011DB27D]
	(No symbol) [0x011D8208]
	(No symbol) [0x011D83A9]
	(No symbol) [0x011CAAC0]
	BaseThreadInitThunk [0x7757FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x776C82AE+286]
	RtlGetAppContainerNamedObjectPath [0x776C827E+238]


In [3]:
pip install undetected_chromedriver

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for undetected_chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47130 sha256=f8c30d8faa0411a98e114389dfaa5628353c737d3183f229d84582a330e82a80
  Stored in directory: c:\users\bedir\appdata\local\pip\cache\wheels\c4\f1\aa\9de6cf276210554d91e9c0526864563e850a428c5e76da4914
Successfully built undetected_chromedriver
Note: you may need to restart the kernel to use updated packages.


In [15]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random
import os

# Satır sonu ve boşlukları temizlemek için
def temizle_metin(text):
    if not isinstance(text, str):
        return text
    return text.replace("\n", " ").replace("\r", " ").strip()


# 📌 CSV'yi sıfırlamak için:
csv_dosya_adi = "istanbulkiralik.csv"
if os.path.exists(csv_dosya_adi):
    os.remove(csv_dosya_adi)  # sadece ilk parça öncesi silinir

# Kullanıcıdan txt dosyası al
txt_file = input("txt dosyası: ")

# Chrome başlat (uc ile)
driver = uc.Chrome()

# txt dosyasındaki her satır için işlemleri yap
with open(txt_file, "r", encoding="utf-8") as file:
    for line in file:
        ilanlar_listesi = []
        line = line.strip().split(",")
        excel_adi = line[0]  # sadece görsel çıktı için
        baslangic = int(line[1])
        bitis = int(line[2])
        base_url = line[3]

        print(f"\n⏳ {excel_adi} için sayfalar geziliyor...")

        ilan_linkleri = []
        for sayfa in range(baslangic, bitis + 1):
            sayfa_url = f"{base_url}?page={sayfa}"
            print(f"🔎 Sayfa {sayfa}: {sayfa_url}")
            driver.get(sayfa_url)

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "a.card-link"))
                )
                kartlar = driver.find_elements(By.CSS_SELECTOR, "a.card-link")
                linkler = [k.get_attribute("href") for k in kartlar]
                ilan_linkleri.extend(linkler)
                print(f"✅ {len(linkler)} ilan bulundu.")
            except:
                print("❌ Sayfa yüklenemedi veya ilan bulunamadı.")

            time.sleep(random.uniform(2.5, 5.0))  # İnsan gibi davranmak için

        print(f"\n📦 Toplam {len(ilan_linkleri)} ilan linki bulundu. Veriler çekiliyor...")

        for i, ilan_url in enumerate(ilan_linkleri, 1):
            print(f"\n📄 {i}. ilan işleniyor: {ilan_url}")
            driver.get(ilan_url)

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "h1"))
                )
            except:
                print("❌ İlan sayfası yüklenmedi.")
                continue

            try:
                baslik = driver.find_element(By.TAG_NAME, "h1").text.strip()
            except:
                baslik = "-"

            try:
                fiyat = driver.find_element(By.CSS_SELECTOR, "p.price").text.strip()
            except:
                fiyat = "-"

            # Konum bilgileri
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "ul.detail-info-location li"))
                )
                konum = driver.find_elements(By.CSS_SELECTOR, "ul.detail-info-location li")
                sehir = konum[0].text.strip() if len(konum) > 0 else "-"
                ilce = konum[1].text.strip() if len(konum) > 1 else "-"
                mahalle = konum[2].text.strip() if len(konum) > 2 else "-"
            except:
                sehir, ilce, mahalle = "-", "-", "-"

            # Teknik bilgiler
            ilan_no = konut_tipi = oda_sayisi = brut_net = banyo_sayisi = kat = kat_sayisi = bina_yasi = isinma = esya = yapi = yapi_durumu = kredi = aidat = tapu = kira = depozito = yakit = cephe = "-"

            try:
                teknik_liste = driver.find_elements(By.CSS_SELECTOR, "ul.adv-info-list li.spec-item")
                for item in teknik_liste:
                    text = item.text.strip()
                    if "İlan no" in text:
                        ilan_no = text.replace("İlan no", "").strip()
                    elif "Konut Tipi" in text:
                        konut_tipi = text.replace("Konut Tipi", "").strip()
                    elif "Oda Sayısı" in text:
                        oda_sayisi = text.replace("Oda Sayısı", "").strip()
                    elif "Brüt / Net M2" in text:
                        brut_net = text.replace("Brüt / Net M2", "").strip()
                    elif "Banyo Sayısı" in text:
                        banyo_sayisi = text.replace("Banyo Sayısı", "").strip()
                    elif "Bulunduğu Kat" in text:
                        kat = text.replace("Bulunduğu Kat", "").strip()
                    elif "Kat Sayısı" in text:
                        kat_sayisi = text.replace("Kat Sayısı", "").strip()
                    elif "Bina Yaşı" in text:
                        bina_yasi = text.replace("Bina Yaşı", "").strip()
                    elif "Isınma Tipi" in text:
                        isinma = text.replace("Isınma Tipi", "").strip()
                    elif "Eşya Durumu" in text:
                        esya = text.replace("Eşya Durumu", "").strip()
                    elif "Yapı Tipi" in text:
                        yapi = text.replace("Yapı Tipi", "").strip()
                    elif "Yapının Durumu" in text:
                        yapi_durumu = text.replace("Yapının Durumu", "").strip()
                    elif "Krediye Uygunluk" in text:
                        kredi = text.replace("Krediye Uygunluk", "").strip()
                    elif "Aidat" in text:
                        aidat = text.replace("Aidat", "").strip()
                    elif "Tapu Durumu" in text:
                        tapu = text.replace("Tapu Durumu", "").strip()
                    elif "Kira Getirisi" in text:
                        kira = text.replace("Kira Getirisi", "").strip()
                    elif "Depozito" in text:
                        depozito = text.replace("Depozito", "").strip()
                    elif "Yakıt Tipi" in text:
                        yakit = text.replace("Yakıt Tipi", "").strip()
                    elif "Cephe" in text:
                        cephe = text.replace("Cephe", "").strip()
            except:
                print("⚠️ Teknik bilgiler alınamadı.")

            ilanlar_listesi.append({
                "İlan no": ilan_no,
                "Başlık": baslik,
                "Fiyat": fiyat,
                "Şehir": sehir,
                "İlçe": ilce,
                "Mahalle": mahalle,
                "Konut Tipi": konut_tipi,
                "Oda Sayısı": oda_sayisi,
                "Brüt / Net M2": brut_net,
                "Banyo Sayısı": banyo_sayisi,
                "Kat": kat,
                "Kat Sayısı": kat_sayisi,
                "Bina Yaşı": bina_yasi,
                "Isınma Tipi": isinma,
                "Eşya Durumu": esya,
                "Yapı Tipi": yapi,
                "Yapının Durumu": yapi_durumu,
                "Krediye Uygunluk": kredi,
                "Aidat": aidat,
                "Tapu Durumu": tapu,
                "Kira Getirisi": kira,
                "Depozito": depozito,
                "Yakıt Tipi": yakit,
                "Cephe": cephe,
                "Link": ilan_url
            })

            time.sleep(random.uniform(2.5, 5.0))  # İlanlar arası bekleme

        # 📤 Tüm ilanları CSV'ye append olarak yaz
        df = pd.DataFrame(ilanlar_listesi)
        df = df.applymap(temizle_metin)  # \n ve \r karakterlerini temizle
        write_header = not os.path.exists(csv_dosya_adi)  # başlık satırı sadece ilk sefer yazılır
        df.to_csv(csv_dosya_adi, mode='a', index=False, header=write_header, encoding="utf-8-sig")
        print(f"\n📥 Veriler eklendi: {csv_dosya_adi}")

driver.quit()
print("\n🎉 Tüm işlemler tamamlandı.")


txt dosyası:  linkler.txt



⏳ istanbulsatilik_bolum1 için sayfalar geziliyor...
🔎 Sayfa 1: https://www.hepsiemlak.com/istanbul-kiralik?page=1
✅ 24 ilan bulundu.

📦 Toplam 24 ilan linki bulundu. Veriler çekiliyor...

📄 1. ilan işleniyor: https://www.hepsiemlak.com/istanbul-esenyurt-barbaros-hayrettin-pasa-kiralik/daire/142108-121

📄 2. ilan işleniyor: https://www.hepsiemlak.com/istanbul-esenyurt-mehtercesme-kiralik/daire/142108-104

📄 3. ilan işleniyor: https://www.hepsiemlak.com/istanbul-esenyurt-yesilkent-kiralik/daire/142108-97

📄 4. ilan işleniyor: https://www.hepsiemlak.com/istanbul-bakirkoy-atakoy-7-8-9-10-kisim-kiralik/residence/3814-4616

📄 5. ilan işleniyor: https://www.hepsiemlak.com/istanbul-eyupsultan-mimar-sinan-kiralik/daire/33208-5842

📄 6. ilan işleniyor: https://www.hepsiemlak.com/istanbul-sariyer-uskumrukoy-kiralik/villa/33208-5852

📄 7. ilan işleniyor: https://www.hepsiemlak.com/istanbul-sisli-merkez-kiralik/daire/40208-2394

📄 8. ilan işleniyor: https://www.hepsiemlak.com/istanbul-sariyer-ayaz

C:\Users\bedir\AppData\Local\Temp\ipykernel_15280\3391751397.py:175: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(temizle_metin)  # \n ve \r karakterlerini temizle


IndexError: list index out of range